In [16]:
import numpy as np 
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
import matplotlib.pyplot as plt
import string
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
#import tensorflow
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kirthika\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kirthika\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
df = pd.read_csv('final_dataset.csv')
df

,Unnamed: 0,Tweets,Date,country,compound,Sentiment,month
0,1,These chinese people on campus are so consider...,2020-01-28 23:41:13+00:00,afghanistan,0.5777,positive,1
1,19,So the first month of the new decade went some...,2020-01-30 23:06:34+00:00,albania,0.2732,positive,1
2,20,Coronavirus sounds nice and all but I'm actual...,2020-01-30 22:09:54+00:00,albania,0.2263,positive,1
3,25,Also a good compendium of information on #nCoV...,2020-01-22 23:19:57+00:00,albania,0.4404,positive,1
4,26,OMG I WAS JUST\r\n\r\nearlier today I was expl...,2020-01-21 23:45:18+00:00,albania,0.9042,positive,1
...,...,...,...,...,...,...,...
168624,36014,"One, so you're saying people only have the rig...",2020-06-19 19:58:00+00:00,united states,0.4490,Positive,6
168625,36015,Kolkata Private hospitals face shortage of bed...,2020-06-19 19:58:00+00:00,india,-0.1372,Negative,6
168626,36016,Reminder: Drive-thru testing is offered at the...,2020-06-19 19:58:00+00:00,united states,0.5106,Positive,6
168627,36017,All Trump supporters should be excited to atte...,2020-06-19 19:57:58+00:00,united states,0.6486,Positive,6


In [18]:
df=df.drop(['Unnamed: 0'],axis=1)

In [19]:
def lower(text):
    #text = text.to_str()
    text = str.lower(text)
    return text
df['Sentiment'] = df['Sentiment'].apply(lambda x: lower(x))
df.head()

,Tweets,Date,country,compound,Sentiment,month
0,These chinese people on campus are so consider...,2020-01-28 23:41:13+00:00,afghanistan,0.5777,positive,1
1,So the first month of the new decade went some...,2020-01-30 23:06:34+00:00,albania,0.2732,positive,1
2,Coronavirus sounds nice and all but I'm actual...,2020-01-30 22:09:54+00:00,albania,0.2263,positive,1
3,Also a good compendium of information on #nCoV...,2020-01-22 23:19:57+00:00,albania,0.4404,positive,1
4,OMG I WAS JUST\r\n\r\nearlier today I was expl...,2020-01-21 23:45:18+00:00,albania,0.9042,positive,1


In [20]:
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

In [21]:
df['Tweets']=df['Tweets'].astype('str')

In [22]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [word for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text
df['Text_cleaned'] = df['Tweets'].apply(lambda x: clean_text(x))

In [23]:
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['Text_stemmed'] = df['Text_cleaned'].apply(lambda x: stemming(x))

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
df['Text_lemmatized_stemmed'] = df['Text_stemmed'].apply(lambda x: lemmatizer(x))
df.head()


,Tweets,Date,country,compound,Sentiment,month,Text_cleaned,Text_stemmed,Text_lemmatized_stemmed
0,These chinese people on campus are so consider...,2020-01-28 23:41:13+00:00,afghanistan,0.5777,positive,1,"[chinese, people, campus, considerate, wear, m...","[chines, peopl, campu, consider, wear, mask, o...","[chine, peopl, campu, consider, wear, mask, ot..."
1,So the first month of the new decade went some...,2020-01-30 23:06:34+00:00,albania,0.2732,positive,1,"[first, month, new, decade, went, something, l...","[first, month, new, decad, went, someth, like,...","[first, month, new, decad, went, someth, like,..."
2,Coronavirus sounds nice and all but I'm actual...,2020-01-30 22:09:54+00:00,albania,0.2263,positive,1,"[coronavirus, sounds, nice, im, actually, hold...","[coronaviru, sound, nice, im, actual, hold, do...","[coronaviru, sound, nice, im, actual, hold, do..."
3,Also a good compendium of information on #nCoV...,2020-01-22 23:19:57+00:00,albania,0.4404,positive,1,"[also, good, compendium, information, ncov, co...","[also, good, compendium, inform, ncov, coronav...","[also, good, compendium, inform, ncov, coronav..."
4,OMG I WAS JUST\r\n\r\nearlier today I was expl...,2020-01-21 23:45:18+00:00,albania,0.9042,positive,1,"[omg, earlier, today, explaining, coronavirus,...","[omg, earlier, today, explain, coronaviru, sit...","[omg, earlier, today, explain, coronaviru, sit..."


In [35]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder() 
#df['deceptive']= le.fit_transform(df['deceptive'])
df['polarity']= le.fit_transform(df['Sentiment'])



In [36]:
y = pd.get_dummies(df['polarity']).values
X = df['Text_lemmatized_stemmed']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

97080     [dear, houthi, abduct, entir, countri, mr, kha...
82276     [usatoday, network, tweet, fact, updat, pandem...
88308     [dyk, peopl, fulli, recov, covid, antibodi, pl...
119009    [fantasi, love, thiswalesnatureweek, ukgovern,...
80584     [impact, coronaviru, shutdown, mean, mani, bus...
                                ...                        
6325      [interest, public, health, infecti, diseas, su...
91310     [logic, question, covid, natur, viru, crazi, c...
91322     [step, daughter, old, bae, scold, slack, schoo...
120554    [princ, joachim, belgium, test, posit, coronav...
20153     [china, coronaviru, outbreak, first, case, rep...
Name: Text_lemmatized_stemmed, Length: 134903, dtype: object

In [37]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

TOP_K = 30000

MAX_SEQUENCE_LENGTH = 10000

def sequence_vectorize(train_texts, val_texts):
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index, max_length

In [38]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [39]:
x_train, x_val = X_train.values, X_test.values
x_train, x_val, word_index, max_length = sequence_vectorize(x_train, x_val)
num_features = min(len(word_index) + 1, TOP_K)

In [40]:

import tensorflow as tf
embedding_vector_length = 256


model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(TOP_K, embedding_vector_length, input_length=max_length),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist1 = model.fit(x_train, y_train, validation_data=(x_val, y_test), epochs=20, batch_size=128)

Train on 134903 samples, validate on 33726 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
134903/134903 [==============================] - 112s 831us/sample - loss: 0.5860 - acc: 0.7725 - val_loss: 0.4259 - val_acc: 0.8527
Epoch 2/20
134903/134903 [==============================] - 115s 854us/sample - loss: 0.4214 - acc: 0.8552 - val_loss: 0.4055 - val_acc: 0.8566
Epoch 3/20
134903/134903 [==============================] - 114s 843us/sample - loss: 0.3769 - acc: 0.8712 - val_loss: 0.4122 - val_acc: 0.8560
Epoch 4/20
134903/134903 [==============================] - 112s 833us/sample - loss: 0.3430 - acc: 0.8818 - val_loss: 0.4115 - val_acc: 0.8565
Epoch 5/20
134903/134903 [==============================] - 111s 825us/sample - loss: 0.3200 - acc: 0.8899 - val_loss: 0.4181 - val_acc: 0.8563
Epoch 6/20
134903/134903 [==============================] - 111s 825us/sample - loss: 0.2990 - acc: 0.8969 - val_loss: 0.4335 - val_acc

In [41]:
accr = model.evaluate(x_val,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

33726/33726 [==============================] - 3s 102us/sample - loss: 0.5739 - acc: 0.8449
Test set
  Loss: 0.574
  Accuracy: 0.845


In [30]:
filename = 'lstm.sav'
pickle.dump(model, open(filename, 'wb'))

TypeError: can't pickle _thread.RLock objects

In [11]:
df[df["Sentiment"] == "neutral"]

,Tweets,Date,country,compound,Sentiment,month,Text_cleaned,Text_stemmed,Text_lemmatized_stemmed
